# Dermoscopic Image Classification 

### Check GPU Status

In [ ]:
!nvidia-smi

In [26]:
!pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.7/11.6 MB 23.5 MB/s eta 0:00:01
   ----- ---------------------------------- 1.5/11.6 MB 15.8 MB/s eta 0:00:01
   ------- -------------------------------- 2.2/11.6 MB 15.3 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/11.6 MB 13.9 MB/s eta 0:00:01
   ----------- ---------------------------- 3.3/11.6 MB 14.9 MB/s eta 0:00:01
   ------------ --------------------------- 3.8/11.6 MB 13.4 MB/s eta 0:00:01
   --------------- ------------------------ 4.4/11.6 MB 14.0 MB/s eta 0:00:01
   ----------------- ---------------------- 4.9/11.6 MB 13.2 MB/s eta 0:00:01
   ------------------ --------------------- 5.5/11.6 MB 13.6 MB/s eta 0:00:01
   --------------------- ------------------ 6.1/11.6 MB 13.5 MB/s eta 0:00:01
   ----------------------- ---------------- 6.7/11.6 MB 13.4 MB/s eta 0:00:01
   ------------------------- -------------- 7.4/11.6 MB 13.6 MB/s eta 0:

In [34]:
# Importing the required libraries
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models

Define hyperparameters

In [51]:
batch_size = 4
img_size = 84
num_classes = 8
data_dir = 'data'
sgd_lr = 0.001
sgd_momentum = 0.9
num_epochs = 100

Checking if GPU is available

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cuda


In [10]:
# Define data transformations
data_transform = {
    'train': transforms.Compose([
        # transforms.Resize((84, 84)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        # transforms.Resize((84, 84)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

In [12]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transform[x]) for x in ['train', 'test']}

In [17]:
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
print(class_names)

{'train': 24499, 'test': 800}
['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']


In [52]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # Calculate the size of the tensor after the last pooling layer
        self.fc_size = 256 * (img_size // 16) * (img_size // 16)

        self.fc1 = nn.Linear(self.fc_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = self.pool(nn.functional.relu(self.conv4(x)))

        x = x.view(-1, self.fc_size)

        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [53]:
model = CNN()

for name, param in model.named_parameters():
    if "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
        
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=sgd_lr, momentum=sgd_momentum)

model = model.to(device)

In [54]:
# Create an array to store the results
results = np.empty((0,5))

for epoch in range(num_epochs):
    start_time = time.time()
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    for phase in ['train', 'test']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criteria(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    end_time = time.time()
    epoch_time = end_time - start_time
    print('Time taken for epoch: {:.0f}m {:.0f}s'.format(epoch_time // 60, epoch_time % 60))

    # Append the results of this epoch to the array
    results = np.append(results, np.array([[epoch, phase, epoch_loss, epoch_acc.cpu(), epoch_time]]), axis=0)

Epoch 0/99
----------
train Loss: 1.2923 Acc: 0.5406
test Loss: 2.7842 Acc: 0.1713
Time taken for epoch: 0m 19s
Epoch 1/99
----------
train Loss: 1.2003 Acc: 0.5657
test Loss: 2.5760 Acc: 0.2075
Time taken for epoch: 0m 19s
Epoch 2/99
----------
train Loss: 1.1668 Acc: 0.5739
test Loss: 2.7522 Acc: 0.2038
Time taken for epoch: 0m 19s
Epoch 3/99
----------
train Loss: 1.1429 Acc: 0.5834
test Loss: 2.5624 Acc: 0.2162
Time taken for epoch: 0m 20s
Epoch 4/99
----------
train Loss: 1.1250 Acc: 0.5899
test Loss: 2.6117 Acc: 0.2300
Time taken for epoch: 0m 19s
Epoch 5/99
----------
train Loss: 1.1108 Acc: 0.5943
test Loss: 2.5684 Acc: 0.2213
Time taken for epoch: 0m 20s
Epoch 6/99
----------
train Loss: 1.0979 Acc: 0.6021
test Loss: 2.5645 Acc: 0.2075
Time taken for epoch: 0m 20s
Epoch 7/99
----------
train Loss: 1.0862 Acc: 0.6057
test Loss: 2.4854 Acc: 0.2450
Time taken for epoch: 0m 21s
Epoch 8/99
----------
train Loss: 1.0788 Acc: 0.6062
test Loss: 2.5828 Acc: 0.2137
Time taken for epoch:

In [55]:
# Save the model
torch.save(model.state_dict(), f'model_{num_epochs}.pth')